In [6]:
from datetime import datetime
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

## 1. Clase de datos y función de extracción de características

In [7]:
class DatosRemitenteViaje:
    def __init__(self, encomienda, viaje):
        self.encomienda = encomienda
        self.viaje = viaje
        self.origen_match = 1 if encomienda["ciudadOrigen"] == viaje["ciudadOrigen"] else 0
        self.destino_match = 1 if encomienda["ciudadDestino"] == viaje["ciudadDestino"] else 0
        fecha_encomienda = datetime.fromisoformat(encomienda["fechaEncomienda"])
        fecha_viaje = datetime.fromisoformat(viaje["fechaViaje"])
        self.dias_diferencia = abs((fecha_viaje - fecha_encomienda).days)

def preparar_features_remitente(datos):
    return [
        datos.origen_match,
        datos.destino_match,
        datos.dias_diferencia
    ]

## 2. Dataset de entrenamiento simulado

In [8]:
data = [
    ({
        "ciudadOrigen": "Bogotá",
        "ciudadDestino": "Medellín",
        "fechaEncomienda": "2025-05-20"
    }, {
        "ciudadOrigen": "Bogotá",
        "ciudadDestino": "Medellín",
        "fechaViaje": "2025-05-21"
    }, 1.0),

    ({
        "ciudadOrigen": "Bogotá",
        "ciudadDestino": "Cali",
        "fechaEncomienda": "2025-05-20"
    }, {
        "ciudadOrigen": "Bogotá",
        "ciudadDestino": "Medellín",
        "fechaViaje": "2025-05-30"
    }, 0.4)
]

## 3. Preparación de datos y entrenamiento

In [9]:
X, y = [], []
for encomienda, viaje, afinidad in data:
    datos = DatosRemitenteViaje(encomienda, viaje)
    X.append(preparar_features_remitente(datos))
    y.append(afinidad)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

modelo = RandomForestRegressor(n_estimators=100, random_state=42)
modelo.fit(X_scaled, y)

joblib.dump(modelo, "modelo_remitente.pkl")
joblib.dump(scaler, "scaler_remitente.pkl")

['scaler_remitente.pkl']

## 4. Predicción de prueba

In [10]:
nueva_encomienda = {
    "ciudadOrigen": "Bogotá",
    "ciudadDestino": "Medellín",
    "fechaEncomienda": "2025-05-20"
}
nuevo_viaje = {
    "ciudadOrigen": "Bogotá",
    "ciudadDestino": "Medellín",
    "fechaViaje": "2025-05-21"
}

datos_nuevos = DatosRemitenteViaje(nueva_encomienda, nuevo_viaje)
features_nuevas = preparar_features_remitente(datos_nuevos)
scaled = scaler.transform([features_nuevas])
pred = modelo.predict(scaled)

print("🔍 Afinidad estimada:", round(float(pred[0]), 3))

🔍 Afinidad estimada: 0.868
